# Decision Trees

## Outline
1. [Training & Visualizing Decision Tree](#part1)
2. [Estimating Class Probabilities](#part2)
3. [Cart Training Algorithm](#part3)
4. [Analysis](#part4)
5. [Gini Impurity or Entropy](#part5)
6. [Regularization Hyperparameters](#part6)
7. [Regression](#part7)

## <a id='part1'>Part 1</a>



In [1]:
from sklearn.datasets import load_iris
from sklearn.tree import DecisionTreeClassifier

iris = load_iris()
X = iris.data[:,2:] # the third column
y = iris.target

tree_clf = DecisionTreeClassifier(max_depth=2)
tree_clf.fit(X,y)

DecisionTreeClassifier(class_weight=None, criterion='gini', max_depth=2,
            max_features=None, max_leaf_nodes=None,
            min_impurity_decrease=0.0, min_impurity_split=None,
            min_samples_leaf=1, min_samples_split=2,
            min_weight_fraction_leaf=0.0, presort=False, random_state=None,
            splitter='best')

In [2]:
# Visualizing purposes 

from sklearn.tree import export_graphviz

export_graphviz(
    tree_clf,
    out_file='iris_tree.dot',
    feature_names = iris.feature_names[2:],
    class_names = iris.target_names,
    rounded=True,
    filled=True
)

After obtaining .dot file, we can convert them into pdf or png.  
```
dot -Tpng iris_tree.dot -o iris_tree.png
```
Result would look like:  

<img src='iris_tree.png'>

__How to read the Figure above__:

Given an instance (with its respective features) we start from the root node. Compare the flower's petal length. If it is smaller than 2.45cm, follow the True option. If it leaf node already, then look at the class represented (setosa).

If length is greater than 2.45 instead, then follow the False route. If not leaf node, keep asking again to compare with the condition, and do the same thing as before. 

One advantage of doing decision tree is that features don't need to be scaled. And that they require little to no data preparation.

__Attributes of each node__:  

_samples_: how many training instances it applies to. For example, there are 100 instances with petal length greater than 2.45 cm, and among those 54 have petal width smaller than 1.75 cm.

_value_: Number of training instances of each class this node applies to. Say for the node in bottom right, there are 0 Iris-Setosa, 1 Iris-Versicolor, and 45 Iris-Virginica.  

_gini_: measure impurity of a node, where impurity means that gini index is 0 (where all training instances it applies to belong to the same class). 

Calculating impurity:  
$$
G_i = 1 - \sum_{k=1}^{n}p_{i,k}^2
$$  
where $p_{i,k}$ is the ratio of class k instances among the training isntance of the i<sup>th</sup> node.  
    
0 Impurity means that like left node of depth 1, all 50 of the instances are considered as Iris-Setosa.  

Scikit learn sues the CART algorithm where it produces binary trees. But there may be other algorithms like ID3 that can produce ternary trees.  

<img src='img/005_1.jpg'>  

This is the graph that shows the decision boundaries for the trained Decision Tree. Thick vertical line represents decision boundary for root node. Thinner lines, means that it is impure. 

## <a id='part2'>Estimating Class Probabilities</a>  

It can also estimate the probability that an instance belongs to a particular class k. For example, a flower with petal length 5 cm and petal width 1.5 cm. It should belong to the depth-2 left node. We look in the value attribute to see how many instances are classified into the 3 classes. 0/54 Iris-Setosa (0%), 49/54 Iris-Versicolor(90.7%), 5/54 Iris-Virginica (9.3%). When told to classify therefore, then this should belong to one with highest probability which we know is the Iris-Versicolor.  


In [3]:
tree_clf.predict_proba([[5,1.5]]) # result should match the above

array([[0.        , 0.90740741, 0.09259259]])

In [4]:
tree_clf.predict([[5,1.5]])

array([1])

So with the above code it shows that our conclusion is proven to be true.

## <a id='part3'>Cart Training Algorithm</a>

CART stands for _Classification and Regression Tree_. Algorihtm first splits the training set into 2 subsets using single feature k and a threshold $t_k$. It chooses the $k$ and $t_k$ by looking at the pair that produces the purest subsets.  

$$
J(k,t_k) = \frac{m_{left}}{m}G_{left} + \frac{m_{right}}{m}G_{right}
$$  
where:  
$G_{left/right}$ measures impurity of left/right subset  
$m_{left/right}$ is the number of instance in the left/right subset.  
Function J is the cost function for classification.  

After successfully split the training set in two, t recursively split the subsets again with same logic. It will stop until it reaches the max depth (which is determined by max_depth parameter), or if it cannot find a split that will reduce impurity.  

Finding optimal tree is an _NP-Complete_ problem. 

## <a id='part4'>Analysis</a>

Assuming that training has been done and the decision tree is formed, we traverse decision tree from root to the leaf. For CART algorithm, it requires going through $O(\log_2{m})$ nodes. For each traversal we only require the comparison between values, it takes roughly O(1) each. In summary therefore, the prediction complexity takes $O(\log_2{m})$.  

As for the training complexity, because the algorithm requires to compare all features on all samples at each node, the complexity woudl be $O(n\times m \log{m})$. n is number of features and _m_ is the number of training instances.

## <a id='part5'>Gini Impurity or Entropy</a>

Previously, we use the gini index as a measure of impurity. However, we can also use entropy as measure. We can control this using the _DecisionTreeClassifier_'s hyperparameter: _criterion_. When all messages are identical, the entropy is zero. 

The measure is:  

$$
H_i = - \sum_{k=1}^{n} p_{i,k} \log{(p_{i,k})} 
$$
where $p_{i,k} \neq 0 $.  

So for example from the decision tree figure above, the entropy for depth-2 left node is $-\frac{49}{54}log{\frac{49}{54}} - \frac{5}{54}\log{\frac{5}{54}}$.  

Gini impurity is slightly faster to compute but tends to isolate the most frequent class in its own branch of the tree, while entropy tends to produce slightly more balanced trees.

## <a id='part6'>Regularization Hyperparameters</a>  

Decision Trees make very little assumptions about training data. If left unconstrained, tree structure will adapt itself to the training data, and likely to cause overfitting. The model is then called nonparametric model because number of parameters is not determined prior to training.  

So to avoid overfitting, we restrict the freedom of Decision Tree during training by regularization. The regularization hyperparameters depend on the algorithm used.  

In general, we can restrict the maximum depth (hyperparameter `max_depth`). Reduce `max_depth` will regularize the model and thus reduce the risk of overfitting.  

Other parameters also include: `min_samples_splot`, `min_samples_leaf`, `min_weight_fraction_leaf`, `max_leaf_nodes`, `max_features`. Increasing the `min_` hyperparameters or reduc

## <a id='part7'>Regression</a>  

We can do regression using decision tree by [Decision Tree Regressor](https://scikit-learn.org/stable/modules/generated/sklearn.tree.DecisionTreeRegressor.html).

In [5]:
from sklearn.tree import DecisionTreeRegressor

tree_reg = DecisionTreeRegressor(max_depth=2)
tree_reg.fit(X,y)

DecisionTreeRegressor(criterion='mse', max_depth=2, max_features=None,
           max_leaf_nodes=None, min_impurity_decrease=0.0,
           min_impurity_split=None, min_samples_leaf=1,
           min_samples_split=2, min_weight_fraction_leaf=0.0,
           presort=False, random_state=None, splitter='best')

In [7]:
export_graphviz(
    tree_reg,
    out_file='regressor_tree.dot',
    feature_names = iris.feature_names[2:],
    class_names = iris.target_names,
    rounded=True,
    filled=True
)